# Cluster

## Objectives

* Fit and evaluate a cluster model to group similar customer behaviour
* Understand profile for each cluster


## Inputs

* outputs/datasets/collection/TelcoCustomerChurn.csv
* instructions on which variables to use for data cleaning and feature engineering. They are found on its respectives notebooks.

## Outputs

* Cluster Pipeline
* Train Set
* Feature importance plot
* Clusters Description
* Cluster Silhouette


---

# Change working directory

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [ ]:
import os
current_dir = os.getcwd()
current_dir

We want to make the parent of the parent of current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [ ]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

Confirm the new current directory

In [ ]:
current_dir = os.getcwd()
current_dir

---

# Load Data for Modelling

In [ ]:
import pandas as pd
df = (pd.read_csv("outputs/datasets/collection/TelcoCustomerChurn.csv")
      .drop(['customerID', 'TotalCharges', 'Churn', 'tenure' ],axis=1) 
)
df.head(3)

# Cluster Pipeline considering all data: KMeans

## ML pipeline for Data Cleaning and Feature Engineering

In [ ]:
from sklearn.pipeline import Pipeline

### Feature Engineering
from feature_engine.encoding import OrdinalEncoder

### PCA
from sklearn.decomposition import PCA

### Feat Selection
from sklearn.feature_selection import SelectFromModel

### Feat Scaling
from sklearn.preprocessing import StandardScaler

### ML algorithms 
from sklearn.cluster import KMeans


def PipelineDataCleaningAndFeatureEngineering():
  pipeline_base = Pipeline(
      [
       
      ("OrdinalCategoricalEncoder",OrdinalEncoder(encoding_method='arbitrary',
                                                  variables = [ 'gender', 'Partner', 'Dependents', 'PhoneService',
                                                               'MultipleLines', 'InternetService', 'OnlineSecurity',
                                                               'OnlineBackup','DeviceProtection', 'TechSupport', 
                                                               'StreamingTV', 'StreamingMovies','Contract', 
                                                               'PaperlessBilling', 'PaymentMethod'])
      ),
    ]
  )

  return pipeline_base

## ML Pipeline for Cluster

where `n_components` of PCA and `n_clusters` of KMeans will be updated

In [ ]:
def PipelineCluster():
  pipe = PipelineDataCleaningAndFeatureEngineering()
  pipe.steps.append(["PCA",PCA(n_components=3, random_state=0)])
  pipe.steps.append(["scaler",StandardScaler()])
  pipe.steps.append(["model",KMeans(n_clusters=4, random_state=0)])
  return pipe

PipelineCluster()

## ML Pipeline for a classifier to explain the clusters

We are considering a model that typically offers good results and features importance can be assessed with `.features_importance_`

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
def PipelineClf2ExplainClusters():
   pipe = PipelineDataCleaningAndFeatureEngineering()
   pipe.steps.append(["feat_selection",SelectFromModel(GradientBoostingClassifier(random_state=0))])
   pipe.steps.append(["scaler",StandardScaler()])
   pipe.steps.append(["model",GradientBoostingClassifier(random_state=0)])
   return pipe
  
PipelineClf2ExplainClusters()

## Principal Component Analysis (PCA)

Apply PCA separately to find the most suitable `n_components`, update the value on ML Pipeline for Cluster

It needs the dataset after data cleaning and feature engineering
  * That means you have to remove 3 steps

In [ ]:
pipeline_pca = PipelineDataCleaningAndFeatureEngineering()
df_pca = pipeline_pca.fit_transform(df)
print(df_pca.shape)
df_pca.head(3)

Apply PCA component

In [ ]:
import numpy as np
from sklearn.decomposition import PCA

n_components = 3

pca = PCA(n_components=n_components).fit(df_pca)
x_PCA = pca.transform(df_pca) # array with transformed PCA

ComponentsList = ["Component " + str(number) for number in range(n_components)]
dfExplVarRatio = pd.DataFrame(
    data= np.round(100 * pca.explained_variance_ratio_ ,2),
    index=ComponentsList,
    columns=['Explained Variance Ratio (%)'])

PercentageOfDataExplained = dfExplVarRatio['Explained Variance Ratio (%)'].sum()

print(f"* The {n_components} components explain {round(PercentageOfDataExplained,2)}% of the data \n")
print(dfExplVarRatio)

Heatmap: PCA components and variables

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")

df_comp = pd.DataFrame(pca.components_, columns=df_pca.columns)
plt.figure(figsize=(20,5))
sns.heatmap(df_comp,center=0,linewidths=.5)
plt.show()

## Elbow Analysis

Find the most suitable `n_clusters`, update the value on ML Pipeline for Cluster


Prepare data for analysis
  * You need to clean and feature engineer your data using the pipeline without the model

In [ ]:
pipeline_cluster = PipelineCluster()
pipeline_elbow = Pipeline(pipeline_cluster.steps[:-1])
df_elbow = pipeline_elbow.fit_transform(df)

print(df_elbow.shape,'\n')
df_elbow

Elbow Analysis

In [ ]:
from yellowbrick.cluster import KElbowVisualizer
from sklearn.cluster import KMeans

visualizer = KElbowVisualizer(KMeans(random_state=0), k=(1,11))
visualizer.fit(df_elbow) 
visualizer.show() 
plt.show()

## Fit Cluster Pipeline

Quick recap in our raw dataset

In [ ]:
print(df.shape)
df.head(3)

Fit Cluster pipeline

In [ ]:
X = df.copy()
pipeline_cluster = PipelineCluster()
pipeline_cluster.fit(X)

Cluster model output is an array with clusters labels

In [ ]:
pipeline_cluster['model'].labels_

## Add cluster labels to dataset

The goal is to merge cluster labels to `X` DataFrame. There is one attention point: **does the pipeline have a step for dropping rows?** 
  * If yes, before merging, we need to drop these rows from X. the code below can do that

```
drop_imputer = DropMissingData(variables =['place here the variables where you drop rows'])
X = drop_imputer.fit_transform(X)
```
* If no, ignore this step.

Our project doesn't need this step. You can confirm that comparing the length of `X` and the length of `cluster label predictions`


In [ ]:
print(X.shape)
print(pipeline_cluster['model'].labels_.shape)

We add a column "`Clusters`" to the data and check clusters distribution
* Clusters don't look to be imbalanced

In [ ]:
X['Clusters'] = pipeline_cluster['model'].labels_

print(f"* Clusters frequencies \n{ X['Clusters'].value_counts(normalize=True).to_frame().round(2)} \n\n")
X['Clusters'].value_counts().sort_values().plot(kind='bar')
plt.show()

This is how our data look like from now
  * Check the last column: `Clusters`
  * Quick reminder: The data is unprocessed (**no data cleaning or feature engineering applied yet**)

In [ ]:
print(X.shape)
X.head(3)

Here we are saving the cluster predictions for this pipeline to use in a fututre moment. We will get back to that soon

In [ ]:
cluster_anwers_with_all_variables = X['Clusters']
cluster_anwers_with_all_variables

## Evaluate Clusters silhouette

To evaluate clusters silhouete we need:
  * data transformed (transform data in the pipeline wihout model step)

In [ ]:
pipeline_silhouette = Pipeline(pipeline_cluster.steps[:-1])
df_transformed = pipeline_silhouette.transform(df)
df_transformed

Plot silhouttes

In [ ]:
from yellowbrick.cluster import SilhouetteVisualizer
visualizer = SilhouetteVisualizer(Pipeline(pipeline_cluster.steps[-1:])[0] , colors='yellowbrick')

visualizer.fit(df_transformed)
visualizer.show()
plt.show()

In [ ]:
# import matplotlib.pyplot as plt
# import seaborn as sns
# import matplotlib.cm as cm
# import numpy as np
# from sklearn.metrics import silhouette_samples, silhouette_score
# sns.set_style("whitegrid")

# def EvaluateClusterSilhouette(X,n_clusters,cluster_labels):

#   print(f"  * The silhouette score range is -1 to +1, where: \n"
#         f"  - 1 means the clusters are dense and properly separated.\n"
#         f"  - 0 means the clusters are overlapping. \n"
#         f"  - A negative score means that these data points from that cluster may be wrong, "
#         f"they should belong to other cluster.")  
  
#   print(f"\n* You should evaluate:\n"
#       "  * If there are clusters with below average silhouette scores. \n"
#       "  * If there is broad variation in the silhouette plots's size across clusters. \n"
#       "  * If the thickness of the silhouettes are uniform/similar in general \n")
  
#   fig = plot_clusters_silhouette(X,n_clusters,cluster_labels)
#   plt.show()

# def plot_clusters_silhouette(X,n_clusters,cluster_labels):

#   silhouette_avg = silhouette_score(X, cluster_labels,random_state=0)

#   fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(15,7))
#   axes.set_xlim([-0.1, 1])
#   axes.set_ylim([0, len(X) + (n_clusters + 1) * 10])

#   sample_silhouette_values = silhouette_samples(X, cluster_labels)
#   y_lower = 10
#   for i in range(n_clusters):
#     ith_cluster_silhouette_values = \
#       sample_silhouette_values[cluster_labels == i]
#     ith_cluster_silhouette_values.sort()
#     size_cluster_i = ith_cluster_silhouette_values.shape[0]
#     y_upper = y_lower + size_cluster_i
#     color = cm.nipy_spectral(float(i) / n_clusters)
#     axes.fill_betweenx(np.arange(y_lower, y_upper),
#               0, ith_cluster_silhouette_values,
#               facecolor=color, edgecolor=color, alpha=0.7)
#     axes.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))
#     y_lower = y_upper + 10

#   axes.set_title("The silhouette plot for each cluster")
#   axes.set_xlabel("The silhouette coefficient values")
#   axes.set_ylabel("Cluster label")
#   axes.text(x=silhouette_avg*1.01, y=len(X)*0.95, s=f"Silhouette Average: {round(silhouette_avg,2)}", fontsize=12, c='r')
#   axes.axvline(x=silhouette_avg, color="red", linestyle="--")
#   axes.set_yticks([])
#   axes.set_xticks([-0.1, 0, 0.2, 0.4, 0.6, 0.8, 1])
#   return fig


## Fit a classifier, where target is cluster labels and features remaining variables

We are in a moment where we have predictions from the cluster pipeline, but we don't have a meaning for them yet. 
* We seek to understand clusters' profile **bold text**

We need to find the most relevant variables, to define each cluster in terms of each relevant variable
* Our new dataset has `Clusters`, which will be the **target for a classifier**. The most relevant features for this classifier, will be the most relevant variables when we run a classifier where the target is the cluster labels!

In [ ]:
X.head()

We copy `X` to a DataFrame `df_clf`, just to separate the cases

In [ ]:
df_clf = X.copy()
print(df_clf.shape)
df_clf.head(3)

Split Train and Test sets

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test,y_train, y_test = train_test_split(
                                    df_clf.drop(['Clusters'],axis=1),
                                    df_clf['Clusters'],
                                    test_size=0.2,
                                    random_state=0
                                    )

print(X_train.shape, X_test.shape)

Create pipeline

In [ ]:
pipeline_clf_cluster = PipelineClf2ExplainClusters()
pipeline_clf_cluster

Fit pipeline

In [ ]:
pipeline_clf_cluster.fit(X_train, y_train)

## Evaluate classifier performance on Train and Test Sets

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_train, pipeline_clf_cluster.predict(X_train)))

In [ ]:
print(classification_report(y_test, pipeline_clf_cluster.predict(X_test)))

## Assess Most Important Features that define a cluster

In [ ]:
# after data cleaning and feat engine, the feature space changes
columns_after_data_cleaning_feat_eng = (PipelineDataCleaningAndFeatureEngineering()
                                        .fit_transform(X_train)
                                        .columns)

best_features = columns_after_data_cleaning_feat_eng[pipeline_clf_cluster['feat_selection'].get_support()].to_list()

# create DataFrame to display feature importance
df_feature_importance = (pd.DataFrame(data={
          'Feature': columns_after_data_cleaning_feat_eng[pipeline_clf_cluster['feat_selection'].get_support()],
          'Importance': pipeline_clf_cluster['model'].feature_importances_})
  .sort_values(by='Importance', ascending=False)
  )

best_features = df_feature_importance['Feature'].to_list() # reassign best features in importance order

# Most important features statement and plot
print(f"* These are the {len(best_features)} most important features in descending order. "
      f"The model was trained on them: \n{best_features} \n")
df_feature_importance.plot(kind='bar',x='Feature',y='Importance')
plt.show()

We will store the best_features for future usage. We will get back to that soon

In [ ]:
best_features_pipeline_all_variables = best_features
best_features_pipeline_all_variables

## Cluster Analysis

Custom Functions for Cluster Analysis

* Table with description for all Clusters

In [ ]:
def Clusters_IndividualDescription(EDA_Cluster,cluster):

  ClustersDescription = pd.DataFrame(columns=EDA_Cluster.columns)
  for col in EDA_Cluster.columns:
    
    try:  # eventually a given cluster will have only mssing data for a given variable
      
      if EDA_Cluster[col].dtypes == 'object':
        
        top_frequencies = EDA_Cluster.dropna(subset=[col])[[col]].value_counts(normalize=True).nlargest(n=3)
        Description = ''
        
        for x in range(len(top_frequencies)):
          freq = top_frequencies.iloc[x]
          category = top_frequencies.index[x][0]
          CategoryPercentage = int(round(freq*100,0))
          statement =  f"'{category}': {CategoryPercentage}% , "  
          Description = Description + statement
        
        ClustersDescription.at[0,col] = Description[:-2]


      
      elif EDA_Cluster[col].dtypes in ['float', 'int']:
        DescStats = EDA_Cluster.dropna(subset=[col])[[col]].describe()
        Q1 = int(round(DescStats.iloc[4,0],0))
        Q3 = int(round(DescStats.iloc[6,0],0))
        Description = f"{Q1} -- {Q3}"
        ClustersDescription.at[0,col] = Description
    
    
    except Exception as e:
      ClustersDescription.at[0,col] = 'Not available'
      print(f"** Error Exception: {e} - cluster {cluster}, variable {col}")
  
  ClustersDescription['Cluster'] = str(cluster)
  
  return ClustersDescription


def DescriptionAllClusters(df_cluster_profile):

  DescriptionAllClusters = pd.DataFrame(columns=df_cluster_profile.drop(['Clusters'],axis=1).columns)
  for cluster in df_cluster_profile.sort_values(by='Clusters')['Clusters'].unique():
    
      EDA_ClusterSubset = df_cluster_profile.query(f"Clusters == {cluster}").drop(['Clusters'],axis=1)
      ClusterDescription = Clusters_IndividualDescription(EDA_ClusterSubset,cluster)
      DescriptionAllClusters = DescriptionAllClusters.append(ClusterDescription)

  
  DescriptionAllClusters.set_index(['Cluster'],inplace=True)
  return DescriptionAllClusters


* Cluster distribution per Variable (absolute and relative)

In [ ]:
import plotly.express as px
def cluster_distribution_per_variable(df,target):


  df_bar_plot = df.value_counts(["Clusters", target]).reset_index() 
  df_bar_plot.columns = ['Clusters',target,'Count']
  df_bar_plot[target] = df_bar_plot[target].astype('object')

  print(f"Clusters distribution across {target} levels")
  fig = px.bar(df_bar_plot, x='Clusters',y='Count',color=target,width=800, height=500)
  fig.update_layout(xaxis=dict(tickmode= 'array',tickvals= df['Clusters'].unique()))
  fig.show()


  df_relative = (df
                 .groupby(["Clusters", target])
                 .size()
                 .groupby(level=0)
                 .apply(lambda x:  100*x / x.sum())
                 .reset_index()
                 .sort_values(by=['Clusters'])
                 )
  df_relative.columns = ['Clusters',target,'Relative Percentage (%)']
 

  print(f"Relative Percentage (%) of {target} in each cluster")
  fig = px.line(df_relative, x='Clusters',y='Relative Percentage (%)',color=target,width=800, height=500)
  fig.update_layout(xaxis=dict(tickmode= 'array',tickvals= df['Clusters'].unique()))
  fig.update_traces(mode='markers+lines')
  fig.show()
 


---

We will study the profile for the main variables that define a cluster


In [ ]:
df_cluster_profile = df_clf.copy()
df_cluster_profile = df_cluster_profile.filter(items=best_features + ['Clusters'], axis=1)
df_cluster_profile.head(3)

Load Churn levels

In [ ]:
df_churn = pd.read_csv("outputs/datasets/collection/TelcoCustomerChurn.csv").filter(['Churn'])
df_churn['Churn'] = df_churn['Churn'].astype('object')
df_churn.head(3)

### Cluster profile on most important features

Considering `df_cluster_profile` and `df_churn`

In [ ]:
pd.set_option('display.max_colwidth', None)
clusters_profile = DescriptionAllClusters(pd.concat([df_cluster_profile,df_churn], axis=1))
clusters_profile

### Clusters distribution across Churn levels & Relative Percentage of Churn in each cluster

In [ ]:
df_cluster_vs_churn=  df_churn.copy()
df_cluster_vs_churn['Clusters'] = X['Clusters']
cluster_distribution_per_variable(df=df_cluster_vs_churn, target='Churn')

# Fit New Cluster Pipeline only on most important features

In order to reduce feature space, we will study the trade-off between considering the previous Cluster Pipeline (fitted with all variables) and creating a new Cluster Pipeline with the variables that are most important to define the clusters from previous pipeline

In [ ]:
best_features_pipeline_all_variables

## Define trade-off and metrics to compare new and previous Cluster Pipeline

To evaluate this tradeoff we will
1. Conduct a PCA analysis, with the same amount of components from 
previous study, in a dataset only with `best_features_pipeline_all_variables` and see if all variables are relevant
2. Conduct a elbow study and check if the same number of clusters is suggested
3. Fit new cluster pipeline and compare if the both clusters predictions are "equivalent"
4. Compare silhoutte score
5. Fit a classifier to explain cluster, and check if performance on Train and Test sets is similar
6. Check if the most important features for the classifier are the same.
7. Compare if the cluster profile from both cases are "equivalent"

If we are happy to say **yes** for them, you can use a cluster pipeline with reduced feature space!
* The **gain** is that in real time (which is the major purpose of Machine Learning) you will need less variables for running predictions and decision making.

## Consider the data with the most relevant variables

In [ ]:
df_reduced = df.filter(best_features_pipeline_all_variables)
df_reduced.head(3)

## Rewrite ML pipeline for Data Cleaning and Feature Engineering

In [ ]:
def PipelineDataCleaningAndFeatureEngineering():
  pipeline_base = Pipeline(
      [
        # we updated the pipeline, considering only the most important variables from previous pipeline      
       ("OrdinalCategoricalEncoder",OrdinalEncoder(encoding_method='arbitrary',
                                                  variables = ['PaymentMethod', 'InternetService',
                                                               'DeviceProtection','OnlineSecurity'])
      ),

    ]
  )

  return pipeline_base

## Apply PCA and compare to previous PCA

It needs the dataset after data cleaning and feature engineering

In [ ]:
pipeline_pca = PipelineDataCleaningAndFeatureEngineering()
df_pca = pipeline_pca.fit_transform(df_reduced)
print(df_pca.shape)
df_pca.head(3)

Apply PCA component

In [ ]:
n_components = 3

pca = PCA(n_components=n_components).fit(df_pca)
x_PCA = pca.transform(df_pca) # array with transformed PCA

ComponentsList = ["Component " + str(number) for number in range(n_components)]
dfExplVarRatio = pd.DataFrame(
    data= np.round(100 * pca.explained_variance_ratio_ ,2),
    index=ComponentsList,
    columns=['Explained Variance Ratio (%)'])

PercentageOfDataExplained = dfExplVarRatio['Explained Variance Ratio (%)'].sum()

print(f"* The {n_components} components explain {round(PercentageOfDataExplained,2)}% of the data \n")
print(dfExplVarRatio)

Heatmap: PCA components and variables

In [ ]:
df_comp = pd.DataFrame(pca.components_, columns=df_pca.columns)
plt.figure(figsize=(20,5))
sns.heatmap(df_comp, center=0, linewidths=.5)
plt.show()

Note that all variables from `best_features_pipeline_all_variables` are indicated as relevant after applying PCA

## Apply Elbow analysis and compare to previous Elbow analysis

Prepare data for analysis
  * You need to clean and feature engineer your data using the pipeline without the model

In [ ]:
pipeline_cluster = PipelineCluster()
pipeline_elbow = Pipeline(pipeline_cluster.steps[:-1])
df_elbow = pipeline_elbow.fit_transform(df_reduced)

print(df_elbow.shape,'\n')
df_elbow

Elbow Analysis

In [ ]:
from yellowbrick.cluster import KElbowVisualizer
from sklearn.cluster import KMeans

visualizer = KElbowVisualizer(KMeans(random_state=0), k=(1,11))
visualizer.fit(df_elbow) 
visualizer.show() 
plt.show()

The same number of clusters is suggested! :)

## Fit New Cluster Pipeline

Quick recap in our raw dataset

In [ ]:
print(df_reduced.shape)
df_reduced.head(3)

Fit Cluster pipeline

In [ ]:
X = df_reduced.copy()
pipeline_cluster = PipelineCluster()
pipeline_cluster.fit(X)

Cluster model output is an array with clusters labels

In [ ]:
pipeline_cluster['model'].labels_

In [ ]:
pipeline_cluster['model'].labels_.shape

## Add cluster labels to dataset

We add a column "Cluster" to the data and check clusters distribution
* Clusters don't look to be imbalanced

In [ ]:
X['Clusters'] = pipeline_cluster['model'].labels_

print(f"* Clusters frequencies \n{ X['Clusters'].value_counts(normalize=True).to_frame().round(2)} \n\n")
X['Clusters'].value_counts().sort_values().plot(kind='bar');

## Compare current cluster labels to previous cluster labels

We just fitted a new cluster pipeline and want to compare if its predictions are "equivalent" from the previous cluster

These are the predictions from **previous** cluster pipeline 

In [ ]:
cluster_anwers_with_all_variables

And these are the predictions from **current** cluster pipeline (trained with `df_reduced`)

In [ ]:
cluster_anwers_with_major_variables = X['Clusters'] 
cluster_anwers_with_major_variables

---

We use a confusion matrix to evaluate if the predictions of both pipelines are **"equivalent"**
* We say equivalent in quotes, because we can't expect that a cluster label 0 in the previous cluster will have the same label in the current cluster. Eventually label 0 in previous cluster pipeline will be a different label in current cluster pipeline
* When we reach this **equivalence**, it means both clusters "clustered" in a similar way but with different labels. And this is fine, since the label itself doesn't have meaning. We will look for the meaning after. 

In [ ]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(cluster_anwers_with_all_variables,
                       cluster_anwers_with_major_variables))

We see that one pipeline labeled 1526 observations as label 2 and other as label 0. 
  * So what is label 2 in the first, is label 0 in the second

We also see one pipeline labeled 1674 observations as label 0 and other as label 2.
  * So what is label 0 in one, is label 2 in the other
  * We notice 146 data points were labelled as 0 in one, and a 1 in the other. This is fine, since is a minority compared to the 1674

When you keep comparing you will notice:
  * What is label 1 in one, is label 3 in the other
  * What is label 3 is one, is label 1 in the other


Conclusion
* We see that both Clusters Pipelines are not predicting the data 100% equivalent, since few data points have different meaning for each pipeline. 
* However this is fine, and we say yes for this criteria. This is part of the trade-off we are up to.

## Evaluate current Clusters silhouette

* To evaluate clusters silhouete we need:
  * data transformed (transform data in the pipeline wihout model step)
  * clusters arrays

In [ ]:
pipeline_silhouette = Pipeline(pipeline_cluster.steps[:-1])
df_transformed = pipeline_silhouette.transform(df_reduced)
df_transformed

Evaluate Cluster Silhouette

In [ ]:
from yellowbrick.cluster import SilhouetteVisualizer
visualizer = SilhouetteVisualizer(Pipeline(pipeline_cluster.steps[-1:])[0] , colors='yellowbrick')

visualizer.fit(df_transformed)
visualizer.show()
plt.show()

The silhoutte score from both pipelines are similar! Now it has even increased a bit :)

## Rewrite ML Pipeline for a classifier to explain the clusters

We want again to explain the major variables for our clusters

We copy `X` to a DataFrame `df_clf`, just to separate the concerns

In [ ]:
df_clf = X.copy()
print(df_clf.shape)
df_clf.head(3)

Split Train and Test sets

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test,y_train, y_test = train_test_split(
                                    df_clf.drop(['Clusters'],axis=1),
                                    df_clf['Clusters'],
                                    test_size=0.2,
                                    random_state=0
                                    )

print(X_train.shape, X_test.shape)

Rewrite pipeline to explain clusters

In [ ]:
def PipelineClf2ExplainClusters():
   pipe = PipelineDataCleaningAndFeatureEngineering()
   # no feature selection step
   pipe.steps.append(["scaler",StandardScaler()])
   pipe.steps.append(["model",GradientBoostingClassifier(random_state=0)])
   return pipe

Create pipeline

In [ ]:
pipeline_clf_cluster = PipelineClf2ExplainClusters()
pipeline_clf_cluster

## Fit a classifier, where target is cluster labels and features remaining variables

In [ ]:
pipeline_clf_cluster.fit(X_train,y_train)

## Evaluate classifier performance on Train and Test Sets

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_train, pipeline_clf_cluster.predict(X_train)))

In [ ]:
print(classification_report(y_test, pipeline_clf_cluster.predict(X_test)))

The performance on Train and Test sets are similar, comparing to the previous pipeline! :)

## Assess Most Important Features

They help the most to define a cluster, compare with previous pipeline
* we can use .feature_importances_ since it is a tree based model
* since we don't have feature selection step, we just get best_features as the Xtrain columns


In [ ]:
best_features = X_train.columns.to_list()

# create DataFrame to display feature importance
df_feature_importance = (pd.DataFrame(data={
    'Feature': X_train.columns,
    'Importance': pipeline_clf_cluster['model'].feature_importances_})
.sort_values(by='Importance', ascending=False)
)

best_features = df_feature_importance['Feature'].to_list()

# Most important features statement and plot
print(f"* These are the {len(best_features)} most important features in descending order. "
      f"The model was trained on them: \n{df_feature_importance['Feature'].to_list()}")

df_feature_importance.plot(kind='bar',x='Feature',y='Importance')
plt.show()

The most relevant variables, in descending order, to explain the cluster from the previous pipeline are

In [ ]:
best_features

We noticed the features importance order are the same from the previous cluster pipeline :)

## Cluster Analysis

We will study the profile for the main variables that define a cluster


In [ ]:
df_cluster_profile = df_clf.copy()
df_cluster_profile = df_cluster_profile.filter(items=best_features + ['Clusters'], axis=1)
df_cluster_profile.head(3)

Load Churn levels

In [ ]:
df_churn = pd.read_csv("outputs/datasets/collection/TelcoCustomerChurn.csv").filter(['Churn'])
df_churn['Churn'] = df_churn['Churn'].astype('object')
df_churn.head(3)

### Cluster profile on most important features

Considering `df_cluster_profile` and `df_churn`

In [ ]:
pd.set_option('display.max_colwidth', None)
clusters_profile = DescriptionAllClusters(pd.concat([df_cluster_profile,df_churn], axis=1))
clusters_profile

### Clusters distribution across Churn levels & Relative Percentage of Churn in each cluster

In [ ]:
df_cluster_vs_churn=  df_churn.copy()
df_cluster_vs_churn['Clusters'] = X['Clusters']
cluster_distribution_per_variable(df=df_cluster_vs_churn, target='Churn')

## Which pipeline should I keep?

Let's recap the criteria we consider to evaluate the **trade-off**
1. Conduct a PCA analysis, with the same amount of components from previous study, in a dataset only with `best_features_pipeline_all_variables` and see if all variables are relevant
2. Conduct a elbow study and check if the same number of clusters is suggested
3. Fit new cluster pipeline and compare if the both clusters predictions are "equivalent"
4. Compare silhoutte score
5. Fit a classifier to explain cluster, and check if performance on Train and Test sets is similar
6. Check if the most important features for the classifier are the same.
7. Compare if the cluster profile from both cases are "equivalent"

We are happy with all criteria above for the new Cluster Pipeline


Now we face a moment of trade-off, where there is no 100% right or wrong decision, it is more a contextual decision
* All 7 criteria support the second pipeline. In addition, there is a great gain to have less variables for predicting live data
* Therefore we have positive evidence to take the pipeline with less variables

In [ ]:
pipeline_cluster

# Push files to Repo


We will generate the following files

* Cluster Pipeline
* Train Set
* Feature importance plot
* Clusters Description
* Cluster Silhouette


In [ ]:
import joblib
import os

version = 'v1'
file_path = f'outputs/ml_pipeline/cluster_analysis/{version}'

try:
  os.makedirs(name=file_path)
except Exception as e:
  print(e)

## Cluster pipeline

In [ ]:
pipeline_cluster

In [ ]:
joblib.dump(value=pipeline_cluster ,
            filename=f"{file_path}/cluster_pipeline.pkl")

## Train Set

In [ ]:
print(df_reduced.shape)
df_reduced.head(3)

In [ ]:
df_reduced.to_csv(f"{file_path}/TrainSet.csv", index=False)

## Most important features plot

These are the features that define a cluster

In [ ]:
df_feature_importance.plot(kind='bar',x='Feature',y='Importance')
plt.show()

In [ ]:
df_feature_importance.plot(kind='bar',x='Feature',y='Importance')
plt.savefig(f"{file_path}/features_define_cluster.png", bbox_inches='tight', dpi=150)

## Cluster Profile

In [ ]:
clusters_profile

In [ ]:
clusters_profile.to_csv(f"{file_path}/clusters_profile.csv")

## Cluster silhouette plot

visualizer = SilhouetteVisualizer(Pipeline(pipeline_cluster.steps[-1:])[0] , colors='yellowbrick')

visualizer.fit(df_transformed)
visualizer.show()
plt.show()

In [ ]:
visualizer = SilhouetteVisualizer(Pipeline(pipeline_cluster.steps[-1:])[0] , colors='yellowbrick')

visualizer.fit(df_transformed)
visualizer.show()
plt.show()

In [ ]:
fig, axes = plt.subplots(figsize=(8,8))
fig = SilhouetteVisualizer(Pipeline(pipeline_cluster.steps[-1:])[0] , colors='yellowbrick', ax=axes)
fig.fit(df_transformed)

plt.savefig(f"{file_path}/clusters_silhouette.png", bbox_inches='tight',dpi=150)

---

## **Push** generated/new files from this Session to GitHub repo

You can push the files now to the Repo!!!!

* Git status

In [ ]:
! git status

* Git commit

In [ ]:
CommitMsg = "added-files-cluster-analysis"
! git add .
! git commit -m {CommitMsg}

* Git Push

In [ ]:
! git push origin main

Good job, clear the cell ouputs and move on to create your dashboard!